In [ ]:
from data_processing_helper import * 
import warnings
warnings.filterwarnings('ignore')

# Import time-frequency functions
from neurodsp.timefrequency import amp_by_time, freq_by_time, phase_by_time
from neurodsp.plts.time_series import plot_time_series, plot_instantaneous_measure


## Specific helper functions

In [ ]:
def getOutputLabelsAndEpochTimes(event_df):
    # Generates the ordered list of output labels and epoch time pairs
    # Input: event_df -- the event dataframe from the csv file
    # Output: 
    #    output_labels -- [1, 2, 4, 3, etc] where the integers correspond to the trial type encoded
    #    epoch_times -- [[<timestamp of start>, <timestamp of end>], [<timestamp of start>, <timestamp of end>], etc]
    
    output_labels = []
    epoch_times = []
    current_epoch = []
    for index, row in event_df.iterrows():
        event_info = row['EventStart'].split("_")
        if event_info[0] == 'start': 
            output_labels.append(int(event_info[1]))
            current_epoch.append(row['time'])
        else :
            current_epoch.append(row['time'])
            epoch_times.append(list(current_epoch))
            current_epoch = []
    return np.array(output_labels), np.array(epoch_times)

In [ ]:
def getEEGEpochs(epoch_times, eeg_df, target_num_trials=1000):
    # Slices and generates the epochs in the eeg_df given the epoch_times
    # Input: 
    #    epoch_times: [[<timestamp of start>, <timestamp of end>], [<timestamp of start>, <timestamp of end>], etc]
    #    eeg_df: dataframe from csv file
    # Output: 
    #    a numpy array containing eeg_epochs (#epoch, #chans, #timepoints)
    eeg_epochs = []
    for epoch_time in epoch_times: 
        sub_df = eeg_df[(eeg_df['time'] > epoch_time[0]) & (eeg_df['time'] < epoch_time[1])]
        sub_df = sub_df.drop(columns=['time'])
        num_above = len(sub_df) - target_num_trials
        if num_above >= 0:
            epoch = np.array(sub_df.values[num_above // 2: len(sub_df) - num_above // 2])[:1000]
            eeg_epochs.append(epoch.T)
            if len(epoch) != 1000:
                print("Warning: Potential off by 1 error. Found trail with != 1000 samples:", len(epoch))
        else: 
            print("Warning: Epoch with less than", target_num_trials, "eeg samples")
    return np.array(eeg_epochs)


In [ ]:
# Spectrogram plotting
def plotSpectrogram_fromEEG(eeg_data, fs=eeg_fs, pre_cut_off_freq=0, post_cut_off_freq=120):
    f, t, Sxx = signal.spectrogram(eeg_data, fs=fs)
    # Calculate the frequency point that corresponds with the desired cut off frequencies
    pre_cut = int(len(f)*(pre_cut_off_freq / f[-1]))
    post_cut = int(len(f)*(post_cut_off_freq / f[-1]))
    plt.pcolormesh(t, f[pre_cut:post_cut], Sxx[pre_cut:post_cut], shading='gouraud')
    plt.colorbar()
    plt.ylabel("Frequency (Hz)")
    plt.xlabel("Time (sec)")

### Main

In [ ]:
## Loading data without EMG
eeg_filename = "./data/self_recorded/eeg_data 15_motorvis.csv"
event_filename = "./data/self_recorded/event_data 15_motorvis.csv"

eeg_chans = ['C4','C2', 'C1', 'C3']
chans = eeg_chans
eeg_df = pd.read_csv(eeg_filename)
eeg_df.columns=['time','C4', 'C2', 'C1', 'C3']

event_df = pd.read_csv(event_filename)
event_df.columns=['time', 'EventStart']
event_types = {0:"eye_close", 1:"left", 2:"right", 3:"foot", 4:"idle"}

# Filter the full data
filtered_df = eeg_df.copy()
for chan in chans:
    filtered_df[chan] = filterEEG(filtered_df[chan].values)

In [ ]:
# ## Loading data with EMG
# eeg_filename = "./data/self_recorded/eeg_data 15_withEMG.csv"
# event_filename = "./data/self_recorded/event_data 15_withEMG.csv"

# eeg_chans = ['C4','C2', 'C1', 'C3']
# chans = ['EMG_R', 'EMG_L', 'VEOG'] + eeg_chans
# eeg_df = pd.read_csv(eeg_filename)
# eeg_df.columns=['time'] + chans

# event_df = pd.read_csv(event_filename)
# event_df.columns=['time', 'EventStart']
# event_types = {0:"eye_close", 1:"left", 2:"right", 3:"foot", 4:"idle"}

# # Filter the full data
# filtered_df = eeg_df.copy()
# for chan in chans:
#     filtered_df[chan] = filterEEG(filtered_df[chan].values)

In [ ]:
filtered_df.head(2)

In [ ]:
event_df.head(2)

In [ ]:
# The sampling rate is not conistent between trials
plt.plot(np.diff(eeg_df['time'])[:100])
plt.axhline(y=1/250, c='black')
plt.title("Time between samples")
plt.ylabel("seconds")
plt.xlabel("timepoint")
plt.show()

In [ ]:
# However, the average sampling rate time difference is close enough to 1/250 within 100 samples. 
np.mean(np.diff(eeg_df['time'])[:100])

In [ ]:
def indexClosest(target_time, eeg_df) :
    start_time = eeg_df['time'][0]
    elems = eeg_df[np.isclose(target_time-start_time, (eeg_df['time']-start_time).values, atol=1e-04)]
    if (len(elems) <= 0): 
        print("Warning: none 1e-04 close")
        elems = eeg_df[np.isclose(target_time-start_time, (eeg_df['time']-start_time).values, atol=1e-03)]
    if (len(elems) <= 0): 
        print("Warning: none 1e-03 close")
        elems = eeg_df[np.isclose(target_time-start_time, (eeg_df['time']-start_time).values, atol=1e-02)]
    if (len(elems) <= 0): 
        print("Warning: none 1e-02 close")
        return -1
    else: 
        print("len elems", len(elems))
        return elems.iloc[0].name



In [ ]:
# Process dfs to get labels, raw eeg epochs, epochs of filtered eeg data, filtered epoch data
output_labels, epoch_times = getOutputLabelsAndEpochTimes(event_df)
raw_eeg_epochs = getEEGEpochs(epoch_times, eeg_df) # Raw eeg epochs
filtered_epochs = getEEGEpochs(epoch_times, filtered_df) # Epoched after filtering

# Create DataFrames
raw_eeg_epoch_df = getDF(raw_eeg_epochs, output_labels, epoch_times, chans)
filtered_epoch_df = getDF(filtered_epochs, output_labels, epoch_times, chans)
filtered_epoch_df.head(20)

In [ ]:
# Get PSD averages for each channel for each event type (0=left or 1=right)
psd_averages_by_type = {}

for event_type in event_types.keys(): 
    psds_only_one_type={}
    freqs_only_one_type={}
    for i, row in filtered_epoch_df[filtered_epoch_df["event_type"] == event_type].iterrows(): 
        for ch in eeg_chans: 
            if ch not in psds_only_one_type: 
                psds_only_one_type[ch] = list()
                freqs_only_one_type[ch] = list()
            f, p = getMeanFreqPSD(row[ch])
            psds_only_one_type[ch].append(p)
            freqs_only_one_type[ch].append(f)
    avg_psds_one_type = {}
    for ch in eeg_chans:
        psds_only_one_type[ch] = np.array(psds_only_one_type[ch])
        avg_psds_one_type[ch] = np.mean(psds_only_one_type[ch], axis=0)
    psd_averages_by_type[event_type] = dict(avg_psds_one_type)

In [ ]:
# View Average PSDs
for event_type in event_types.keys(): 
    for ch in eeg_chans[:]: 
        plotPSD(freqs_only_one_type[eeg_chans[0]][0], psd_averages_by_type[event_type][ch],pre_cut_off_freq=2, post_cut_off_freq=30, label=ch)

    plt.legend()
    plt.title("event type: " + event_types[event_type])
    plt.show()

In [ ]:
indexClosest(filtered_epoch_df['start_time'][4], eeg_df)

In [ ]:
# Try adjust these variables to see different time ranges! 
# A single trial is 4 seconds or 1000 timpoints (4 ms per timepoint)
# Hint: refer to the Epoched data dataframe for the time of each trial
start_time_timepoints = indexClosest(filtered_epoch_df['start_time'][8], eeg_df)
end_time_timepoints = start_time_timepoints + 2000 # Specify number of more timepoints we want past start

# Plot a single EEG channel
plt.figure(figsize=(15,5))
plt.plot(filtered_df['C3'].values[start_time_timepoints:end_time_timepoints])
plt.title("C3 -- " + str(start_time_timepoints) + " to " + str(end_time_timepoints))
plt.xlabel("timepoints")
plt.ylabel("Voltage (uV)")
plt.show()

# Plot instananeous alpha amplitude
sig = filtered_df['C3'].values[start_time_timepoints:end_time_timepoints]
times = filtered_df['time'].values[start_time_timepoints:end_time_timepoints] - filtered_df['time'][0]
amp = amp_by_time(sig, eeg_fs, (7, 12))
plot_instantaneous_measure(times, [sig, amp], 'amplitude',
                           labels=['Raw Signal', 'Amplitude'])

# # Plot a single EOG channel
# plt.figure(figsize=(15,5))
# plt.plot(eeg_df['EOG:ch01'].values[start_time_timepoints:end_time_timepoints])
# plt.title("EOG:ch01 -- " + str(start_time_timepoints) + " to " + str(end_time_timepoints))
# plt.xlabel("timepoints")
# plt.ylabel("Voltage (uV)")
# plt.show()

# Plot the PSD of the single EEG channel
plt.figure(figsize=(15,5))
plotPSD_fromEEG(filtered_df['C3'].values[start_time_timepoints:end_time_timepoints], pre_cut_off_freq=2, post_cut_off_freq=40,label="C3")
plt.title("PSD of C3 in the timespan provided")
plt.legend()
plt.show()

# Plot the spectrogram of the single EEG channel
plt.figure(figsize=(15,5))
plotSpectrogram_fromEEG(filtered_df['C3'].values[start_time_timepoints:end_time_timepoints], pre_cut_off_freq=2, post_cut_off_freq=40)
plt.title("Spectrogram of C3 in the timespan provided")
plt.show()



In [ ]:
# Visualize EEG and PSD for one trial
plt.figure(figsize=(15,5))
trial_num = 17

for ch in eeg_chans: 
    plt.plot(filtered_epoch_df[ch][trial_num], label=ch)
plt.ylabel("Voltage (uV)")
plt.xlabel("timepoints @ 250Hz")
plt.title("EEG of one motor imagery trial")
plt.legend() 
plt.show()

plt.figure(figsize=(15,5))
for ch in eeg_chans: 
    plotPSD_fromEEG(filtered_epoch_df.iloc[trial_num][ch], pre_cut_off_freq=0.5, post_cut_off_freq=40, label=ch)
plt.title("PSD of one motor imagery trial")
plt.legend()
plt.show()


In [ ]:
# Get PSD averages for each channel for each event type 
# (0 = eye close, 1 = left, 2 = right, 3 = foot, 4 = idle)
psd_averages_by_type = {}

for event_type in range(0, 5): 
    psds_only_one_type={}
    freqs_only_one_type={}
    for i, row in filtered_epoch_df[filtered_epoch_df["event_type"] == event_type].iterrows(): 
        for ch in chans: 
            if ch not in psds_only_one_type: 
                psds_only_one_type[ch] = list()
                freqs_only_one_type[ch] = list()
            f, p = getMeanFreqPSD(row[ch])
            psds_only_one_type[ch].append(p)
            freqs_only_one_type[ch].append(f)
    avg_psds_one_type = {}
    for ch in chans:
        psds_only_one_type[ch] = np.array(psds_only_one_type[ch])
        avg_psds_one_type[ch] = np.mean(psds_only_one_type[ch], axis=0)
    psd_averages_by_type[event_type] = dict(avg_psds_one_type)

In [ ]:
# View Average PSDs
for event_type in range(0, 5): 
    for ch in ['C4', 'C3']: 
        plotPSD(freqs_only_one_type[chans[0]][0], psd_averages_by_type[event_type][ch],pre_cut_off_freq=2, post_cut_off_freq=30, label=ch)

    # Plot for each event type
    plt.legend()
    plt.title("event type " + event_types[event_type])
    plt.show()

## Power bin feature analysis

In [ ]:
import pyeeg
def getPowerRatio(eeg_data, binning, eeg_fs=250):
    power, power_ratio = pyeeg.bin_power(eeg_data, binning, eeg_fs)
    return np.array(power_ratio)
def getIntervals(binning): 
    intervals = list()
    for i, val in enumerate(binning[:-1]): 
        intervals.append((val, binning[i+1]))
    return intervals

In [ ]:
def plotMultipleBarGraphs(bars, bar_width, bar_names, group_names, error_values=None, title=None, xlabel=None, ylabel=None): 
    if len(bar_names) != len(bars):
        print("group names must be same length as bars")
        return 
    # Set position of bar on X axis
    positions = list()
    positions.append(np.arange(len(bars[0])))
    for i, bar in enumerate(bars): 
        if i>0: 
            positions.append([x + bar_width for x in positions[i-1]])

    # Make the plot
    for i, pos in enumerate(positions):
        plt.bar(pos, bars[i], width=bar_width, label=bar_names[i])
    
    if error_values is not None: 
        for i, pos in enumerate(positions):
            plt.errorbar(pos, bars[i], yerr=error_values[i], fmt='.k')
    
    # Add xticks on the middle of the group bars
    if xlabel: 
        plt.xlabel(xlabel)
    if ylabel: 
        plt.ylabel(ylabel)
    if title: 
        plt.title(title)
    plt.xticks([r + bar_width for r in range(len(bars[0]))], group_names)

    # Create legend & Show graphic
    plt.legend()
    plt.show()

In [ ]:
# Calculate the power ratios of each epoch and each eeg channel
epoched_df_without_calibration = filtered_epoch_df

power_ratios = {'y': []}
sub_binning=[0.5, 4, 7, 12, 30]
sub_intervals = getIntervals(sub_binning)
for i in range(0, len(filtered_epoch_df)): 
    event_type = filtered_epoch_df['event_type'][i]
    for ch in eeg_chans: 
        ratios = getPowerRatio(filtered_epoch_df[ch][i][100:900], sub_binning)
        for j, interval in enumerate(sub_intervals): 
            key = ch + "_" + str(interval)
            if key not in power_ratios: 
                power_ratios[key] = list()
            power_ratios[key].append(ratios[j])
    power_ratios['y'].append(filtered_epoch_df['event_type'][i])

power_ratios_df = pd.DataFrame(power_ratios)
bi_class_df = power_ratios_df[(power_ratios_df['y'] == 1) | (power_ratios_df['y'] == 2)]


In [ ]:
# Calculate the standard error means between epochs for each channel from the power ratios obtained previously
chan_frequency_sems = {}
chan_frequency_avgs = {}

for event_type in event_types: 
    for ch in eeg_chans: 
        for interval in sub_intervals: 
            key = ch + "_" + str(interval)
            if key not in chan_frequency_sems: 
                chan_frequency_sems[key] = list()
                chan_frequency_avgs[key] = list()
            this_data = power_ratios_df[power_ratios_df['y'] == event_type][key]
            sem = np.std(this_data) / np.sqrt(len(this_data))
            chan_frequency_sems[key].append(sem)
            chan_frequency_avgs[key].append(np.mean(this_data))
            
std_err_df = pd.DataFrame(chan_frequency_sems)
avg_df = pd.DataFrame(chan_frequency_avgs)

In [ ]:
# Plot by channel
# rows = intervals, columns = event types

for chan in eeg_chans: 
    chan_of_interest = chan
    event_power_ratios = {}
    event_sems = {}
    power_ratios_for_chan = []
    sem_for_chan = []
    for event_type in range(1, 5): 
        if event_type not in event_power_ratios: 
            event_power_ratios[event_type] = []
            event_sems[event_type] = []
        for interval in sub_intervals: 
            key = chan_of_interest + "_" + str(interval)
            event_power_ratios[event_type].append(avg_df[key][event_type])
            event_sems[event_type].append(std_err_df[key][event_type])

    event_sems_df = pd.DataFrame(event_sems)
    event_power_ratios_df = pd.DataFrame(event_power_ratios)
    
    plt.title(chan_of_interest)

    plotMultipleBarGraphs(np.transpose(np.array(event_power_ratios_df)), 0.15, [1, 2, 3, 4], sub_intervals, error_values=np.transpose(np.array(event_sems_df)))
    
    


# Power Bin Modeling

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut

from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import accuracy_score

In [ ]:
# Abstract class
class myModel:
    def __init__(self):
        pass
    def fit(X, Y):
        pass
    def predict(X):
        pass

In [ ]:
# Extract trials that are for left vs right hand imagery
filtered_epoch_bi_class_df = filtered_epoch_df[(filtered_epoch_df['event_type'] == 1) | (filtered_epoch_df['event_type'] == 2)]
X = filtered_epoch_bi_class_df[eeg_chans].values
Y = filtered_epoch_bi_class_df['event_type'].values

# Split the 4s trials into 2.8s trials. front and end to obtain 2x trials
X_split = []
Y_split = []
for i in range(30): 
    eeg_data = []
    for j in range(4): 
        eeg_data.append(X[i, j][:700])
    eeg_data_2 = []
    for j in range(4): 
        eeg_data_2.append(X[i, j][300:])
    
    X_split.append(np.array(eeg_data))
    X_split.append(np.array(eeg_data_2))
    Y_split.append(Y[i])
    Y_split.append(Y[i])

# Shuffle
X = np.array(X_split)
Y = np.array(Y_split)
temp=list(zip(Y, X))
random.shuffle(temp)
Y_shuffled, X_shuffled = zip(*temp)

# Split train/test
num_train = int(len(X_shuffled)*4/(4+1))
X_train = np.array(X_shuffled[:num_train])
X_test = np.array(X_shuffled[num_train:])
Y_train = np.array(Y_shuffled[:num_train])
Y_test = np.array(Y_shuffled[num_train:])

In [ ]:
# Power Bin Model (may have other decision algorithms)
class PowerBinModel(myModel): # XDAWN Covariance Preprocessing + Linear Regression Classifier
    def __init__(self, chans, num_top=5):
        super().__init__()
        self.mod_types = [SVC, LinearDiscriminantAnalysis, RandomForestClassifier]
        self.mod_type = SVC 
        self.binning = [0.5, 4, 7, 12, 30]
        self.intervals = getIntervals(self.binning)
        self.model = self.mod_type()
        self.scaler = StandardScaler()
        self.chans = chans
        self.feat_names = [str(ch) + "_" + str(ints) for ch in self.chans for ints in self.intervals]
        self.feature_indx = None
        self.num_top = num_top
        
    def getFeatureNames(self):
        return self.feat_names
        
    def _getFeatures(self, eeg_datas) :
        feats = []
        for eeg_data in eeg_datas: 
            feats.extend(getPowerRatio(eeg_data, self.binning))
        return feats
    
    def _selectTopFeatures(self, X_features, Y) :
        # Get the unique classes of Y to be able to separate the features 
        unique_Y = np.unique(Y)
        X_features_mean = []
        for un in unique_Y: 
            X_features_mean.append(np.mean(X_features[Y == un, :], 0))
        X_features_mean_diff = np.diff(X_features_mean, 0)
        #X_features_mean_diff = np.mean(X_features_mean_diff, 0)
        # Get the index of the sorted mean difference array
        args = np.argsort(X_features_mean_diff[0])
        # Reverse args to largest features are first
        args = args[::-1]
        feature_indx = args[:self.num_top]
        X_features = np.transpose([X_features[:, i] for i in feature_indx])
        return X_features, feature_indx

    def fit(self, X, Y):
        self.model = self.mod_type()
        # X shape must be (#Trials, #Chans, #Timepoints)
        # TODO: Add cross validation
        # TODO: Add model stacking to training
        X_features = np.array([self._getFeatures(eeg_datas) for eeg_datas in X])
        X_features, self.feature_indx = self._selectTopFeatures(X_features, Y)
        X_features = self.scaler.fit_transform(X_features)
        self.model.fit(X_features,Y)
    
    def evaluate(self, X, Y): 
        unique_Y = np.unique(Y)
        loo = LeaveOneOut()
        X_features = np.array([self._getFeatures(eeg_datas) for eeg_datas in X])
        accs = [] 
        for mod in self.mod_types: 
            y_pred = []
            y_true = []
            for train_ix, test_ix in loo.split(Y):
                # split data
                X_train_i, X_test_i = X_features[train_ix, :], X_features[test_ix, :]
                y_train_i, y_test_i = Y[train_ix], Y[test_ix]

                X_train_i, feature_indx = self._selectTopFeatures(X_train_i, y_train_i)
                scaler = StandardScaler()
                X_train_i = scaler.fit_transform(X_train_i)

                # fit model
                model = mod()
                model.fit(X_train_i, y_train_i)

                X_test_i = np.transpose([X_test_i[:, i] for i in feature_indx])
                X_test_i = scaler.transform(X_test_i)

                # evaluate model
                yhat = model.predict(X_test_i)
                # store
                y_true.append(y_test_i[0])
                y_pred.append(yhat[0])

            # calculate accuracy
            acc = accuracy_score(y_true, y_pred)
            accs.append(acc)
        print(accs)
        return max(accs), accs.index(max(accs))
        
    def predict(self, X):
        # X shape must be (#Trials, #Chans, #Timepoints)
        X_features = np.array([self._getFeatures(eeg_datas) for eeg_datas in X])
        X_features = np.transpose([X_features[:, i] for i in self.feature_indx])
        X_features = self.scaler.transform(X_features)
        return self.model.predict(X_features)


In [ ]:
# Accuracy on the shuffled train and test split
num_feats_used = 4
power_bin_model = PowerBinModel(eeg_chans, num_top=15)
num_feats = len(power_bin_model.getFeatureNames())
print("num total feats", num_feats)
print("num used", num_feats_used)
power_bin_model.fit(X_train, Y_train)
Y_pred = power_bin_model.predict(X_test)
print(classification_report(Y_test, Y_pred))
print("Accuracy:", sum(Y_test==Y_pred) / len(Y_test))

In [ ]:
# Evaluation of model as a function of number of features used. 
# These accuracies are obtained by training the model on everything except one datapoint and then predicting that datapoint. 
accs = []
indxs = []
for i in range(1,num_feats) :
    power_bin_model = PowerBinModel(eeg_chans, num_top=i)
    acc, inx = power_bin_model.evaluate(X, Y)
    accs.append(acc)
    indxs.append(inx)



In [ ]:
plt.plot(list(range(1,num_feats)), accs)
plt.xlabel("num features")
plt.ylabel("accuracy")
plt.show()

plt.scatter(list(range(1,num_feats)), indxs)
plt.xlabel("num features")
plt.ylabel("which model won")
plt.show()

## XDawn

In [ ]:
# # for models:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace


In [ ]:
# Arrays of eeg data (for XDawnModel)
start_chan = 0 if filtered_epochs.shape[1] == 4 else 3
X = filtered_epochs[3:, start_chan:]
Y = output_labels[3:]

temp=list(zip(Y, X))
random.shuffle(temp)
Y_shuffled, X_shuffled = zip(*temp)

#Split train/test
X_train = np.array(X_shuffled[:int(len(X_shuffled)*4/(4+1))])
X_test = np.array(X_shuffled[int(len(X_shuffled)*4/(4+1)):])
Y_train = np.array(Y_shuffled[:int(len(Y_shuffled)*4/(4+1))])
Y_test = np.array(Y_shuffled[int(len(Y_shuffled)*4/(4+1)):])

In [ ]:


# XDawn
class XDawnLRModel(myModel): # XDAWN Covariance Preprocessing + Linear Regression Classifier
    def __init__(self):
        super().__init__()
        self.XC = XdawnCovariances(nfilter = 1) # the number of filters can be changed
        self.logreg = LogisticRegression()
        
    def fit(self, X, Y):
        X_transformed = self.XC.fit_transform(X, Y)
        X_transformed = TangentSpace(metric='riemann').fit_transform(X_transformed)
        self.logreg.fit(X_transformed,Y)
        
    def predict(self, X):
        X_transformed = self.XC.transform(X)
        X_transformed = TangentSpace(metric='riemann').fit_transform(X_transformed)
        return self.logreg.predict(X_transformed)
    
    def evaluate(self, X, Y): 
        loo = LeaveOneOut()
        X_transformed = self.XC.fit_transform(X, Y)
        X_transformed = TangentSpace(metric='riemann').fit_transform(X_transformed)
        y_pred = []
        y_true = []
        for train_ix, test_ix in loo.split(Y):
            # split data
            X_train, X_test = X_transformed[train_ix, :], X_transformed[test_ix, :]
            y_train, y_test = Y[train_ix], Y[test_ix]
            
            # fit model
            model = LogisticRegression()
            model.fit(X_train, y_train)
            
            # evaluate model
            yhat = model.predict(X_test)
            # store
            y_true.append(y_test[0])
            y_pred.append(yhat[0])
        
        # calculate accuracy
        acc = accuracy_score(y_true, y_pred)
        print('Accuracy: %.3f' % acc)



In [ ]:
model = XDawnLRModel()
model.evaluate(X, Y)

In [ ]:
model = XDawnLRModel()
model.fit(X_train, Y_train)
#return model
Y_pred = model.predict(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
sum(Y_test == Y_pred)/len(Y_test)